In [1]:
import pandas as pd
import numpy as np

In [2]:
dtype_victims={
    'case_id': np.uint64, # cannot use in tsince biggest case id is 9.870011e+18, np.uint64: 64-bit unsigned integer (from 0 to 2**64-1)
    'id': int,
    'party_number': int,
    'victim_age': pd.Int16Dtype(),
    'victim_degree_of_injury': str,# map to ints
    'victim_ejected': pd.Int16Dtype(),
    'victim_role': str,# map to ints
    'victim_safety_equipment_1': str,# map char to ints
    'victim_safety_equipment_2': str,# map char to ints
    'victim_seating_position': str,# pd.Int16Dtype(),# int (use smallest int type)
    'victim_sex': str, #char
}

In [3]:
nrows = 5

###### Delete nrows ######

use converters : Dict of functions for converting values in certain columns. Keys can either be integers or column labels.

??na_values / keep_default_na / na_filter

check with sql if need to infer_datetime_format/keep_date_col/date_parser/dayfirst/cache_dates

comment is it useful?

error_bad_lines/warn_bad_lines

4082685

In [4]:
def victim_degree_of_injury_converter(x) :
    choices = pd.read_csv('tables/Victims/victim_degree_of_injury.csv')
    descriptions = (description.lower() for description in choices['description'])

    if x.lower() in descriptions: # add option to fix typos
        return x.lower() #find a way to return correct casing
    try:
        int(x)
    except: 
        raise Exception('Victim degree of injury should be an int between 0 or 7 or one of the predefined strings. The actual value was: {}'.format(x))

    if int(x) in choices['id']:
        return choices['description'][int(x)].lower()
    
    raise Exception('Victim degree of injury should be an int between 0 or 7 or one of the predefined strings. The actual value was: {}'.format(x))

In [5]:
victim_degree_of_injury_converter('0')

'killed'

In [6]:
# converters = {'victim_degree_of_injury': victim_degree_of_injury_converter}
# usecols= set(dtype_victims.keys()).union(converters.keys())

In [7]:
victims_data = pd.read_csv("CSV-2018/victims2018.csv",dtype=dtype_victims,usecols=dtype_victims.keys()) 

In [107]:
victims_data

,case_id,id,party_number,victim_age,victim_degree_of_injury,victim_ejected,victim_role,victim_safety_equipment_1,victim_safety_equipment_2,victim_seating_position,victim_sex
0,3,3,1,21,no injury,0,2,G,NaN,3,male
1,5,4,1,44,severe injury,0,1,G,NaN,1,male
2,8,5,1,59,no injury,0,2,G,NaN,3,female
3,8,6,2,31,no injury,0,2,G,NaN,3,male
4,9,7,2,14,complaint of pain,0,2,C,NaN,6,female
...,...,...,...,...,...,...,...,...,...,...,...
4082680,9870011226102009803,8817531,2,20,other visible injury,0,1,G,NaN,1,female
4082681,9870011231152508671,8817532,1,27,complaint of pain,0,2,H,NaN,3,female
4082682,9870011231152508671,8817533,2,74,complaint of pain,0,2,G,NaN,3,female
4082683,9870011231152508671,8817534,2,64,complaint of pain,0,1,G,NaN,1,male


In [11]:
clean_data = victims_data.copy()

In [51]:
translation_victim_age = {
    "not stated": 998,
    "pregnancy": 999,
}

translation_victim_degree_of_injury = {
    "killed": 1,
    "severe injury": 2,
    "other visible injury": 3,
    "complaint of pain": 4,
    "suspected serious injury": 5,
    "suspected minor injury": 6,
    "possible injury": 7,
    "no injury": 0,
}

translation_victim_ejected = {
    "not ejected": 0,
    "fully ejected": 1,
    "partially ejected": 2,
    "unknown": 3,
}

translation_victim_role = {
    "driver": 1,
    "passenger (includes non-operator on bicycle or any victim on/in parked vehicle or multiple victims on/in non-motor vehicle)": 2,
    "pedestrian": 3,
    "bicyclist": 4,
    "other (single victim on/in non-motor vehicle; e.g. ridden animal, horse-drawn carriage, train, or building)": 5,
    "non-injured party": 6,
}

translation_victim_sex = {
    "male": 'M',
    "female": 'F',
}

In [52]:
clean_data["victim_age"] = victims_data["victim_age"].map(lambda s: translation_victim_age.get(s, s)).astype(pd.Int16Dtype())
clean_data["victim_degree_of_injury"] = victims_data["victim_degree_of_injury"].map(lambda s: translation_victim_degree_of_injury.get(s, None)).astype(pd.Int16Dtype())
clean_data["victim_ejected"] = victims_data["victim_ejected"].map(lambda s: translation_victim_ejected.get(s, None)).astype(pd.Int16Dtype())
clean_data["victim_role"] = victims_data["victim_role"].map(lambda s: translation_victim_ejected.get(s, None)).astype(pd.Int16Dtype())
clean_data["victim_sex"] = victims_data["victim_sex"].map(lambda s: translation_victim_sex.get(s, None))


why does party number contain 1 ->89

victim age: change 998->not stated
victim_age (how to do dict 

should we make all inputs lower case when reading csv?

victim ejected: why do we have 4 in the set?

where is victim_safety_equipment_2 csv and translation

how to do translation for victim_seating_position

In [47]:
# s = victims_data['victim_age']
s = clean_data['victim_age']

# for x in s:
#     int(x)
    
set(s)



{0,
 1,
 10,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 11,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 12,
 120,
 121,
 122,
 123,
 124,
 125,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 2,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 3,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 4,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 5,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 6,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 7,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 8,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 9,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 999,
 <NA>}

First take memory usage 

Index                             128

case_id                      32661480

id                           32661480

party_number                 32661480

victim_age                   12248055

victim_degree_of_injury      32661480

victim_ejected               12248055

victim_role                  32661480

victim_safety_equipment_1    32661480

victim_safety_equipment_2    32661480

victim_seating_position      32661480

victim_sex                   32661480

dtype: int64

In [108]:
victims_data.memory_usage()

Index                             128
case_id                      32661480
id                           32661480
party_number                 32661480
victim_age                   12248055
victim_degree_of_injury      32661480
victim_ejected               12248055
victim_role                  32661480
victim_safety_equipment_1    32661480
victim_safety_equipment_2    32661480
victim_seating_position      32661480
victim_sex                   32661480
dtype: int64

In [109]:
# def victim_degree_of_injury_converter(x) :
#     choices = pd.read_csv('tables/Victims/victim_degree_of_injury.csv')
#     descriptions = (description.lower() for description in choices['description'])

#     if x in choices['id']:
#         return x
#     if x in choices['description']:
#         return choices[]
    


In [113]:
# victims_data['victim_degree_of_injury']
# choices = pd.read_csv('tables/Victims/victim_degree_of_injury.csv')
# a = choices.to_dict()
# a['description'][0]
# dictd = [choices['id'],choices['id']]
# dictd
set(victims_data['victim_degree_of_injury'].apply(lambda x : return x)

SyntaxError: invalid syntax (<ipython-input-113-28e98e6f997e>, line 7)

In [ ]:
choices = pd.read_csv('tables/Victims/victim_degree_of_injury.csv')
choices

In [ ]:
choices['id']
choices['description']

In [ ]:
set(victims_data['victim_degree_of_injury']).issubset(set(choices))


In [ ]:
set(victims_data['victim_degree_of_injury'])

In [103]:
set(choices).difference(set(victims_data['victim_degree_of_injury']))

{'description', 'id'}

In [104]:
for x in range(len(victims_data['victim_degree_of_injury'])):
    victims_data['victim_degree_of_injury'] = 0

In [105]:
victims_data['victim_degree_of_injury']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
Name: victim_degree_of_injury, dtype: int64